In [1]:
import sys
sys.path.append('..')

import random

# Pynetics QuickStart

In this example we are going to build a very simple and useless algorithm to explore the possibilities of the pynetics library.

Our problem will be as follows. We'll going to develop a genetic algorithm (from now on _GA_) to find what binary list of lenght $L=N$ is the one with the bigger sum. Yes, it's totally absurd, but useful to learn GAs and this library.

Let's start from the begining. The individuals.

## Representing individuals

The individuals are the most important components in a genetic algorithm. Each individual is a possible solution, good or bad, for our problem.

We want to model an individual capable of representing a possible solution to our problem. Pynetics has a perfect representation for this problem, the `BinaryIndividual`, so it's no necessary to create a custom individual. We'll cross that bridge when we get to it.

The algorithm will create individuals using a `SpawningPool` implementation. We're going to use a special implementation inside the module `pynetics.ga_bin` called `BinaryIndividualSpawningPool`, that creates `BinaryIndividual` instances of the given size.

In [2]:
from pynetics.ga_bin import BinaryIndividualSpawningPool

# Let's define the size of our individuals (the numer of 1's and 0's)
individual_size = 25
binary_individual_spawning_pool = BinaryIndividualSpawningPool(size=individual_size)

Now the spawning pool will be capable of creating individuals of the specified size. The genetic algorithm will create a population of individuals using the `spawn` method to populate it. We'll also specify a population size for the algorithm and see an example of population:

In [3]:
population_size = 10

for i in range(population_size):
    individual = binary_individual_spawning_pool.spawn()
    print(i, '->', individual)

0 -> 1000011000011001111000111
1 -> 0110001010101000110001001
2 -> 0100010101100110000011011
3 -> 0110000101110111001001111
4 -> 0100011110000001010100110
5 -> 1001001110110101100001110
6 -> 0011110110001011001111000
7 -> 0110001100001101010001101
8 -> 0001011111101111001001100
9 -> 0111001101100010110100001


## Fitness

Our individuals are solutions for the problem but, ¿how can we measure how good or bad are they? That is what the `fitness` is for. It's a function that will return a float value. The bigger the value, the better the individual is.

We could use a fitness function equals to the sum of all $1$'s but if we want to stop the algorithm based on the fitness, is not the same the best fitness for an individual of size 10 than an individual of size 20.

So the fitness funcion we're gonna use is a function with the form $1 / (1 + \alpha)$, being $\alpha$ the error of our individual. The error will be computed as the number of $0$'s the individual has.

In [4]:
def maximize_ones_fitness(individual):
    error = len(individual) - sum(individual)
    return 1 / (1 + error)

This function guarantees that the fitness will belong to the $(0, 1]$ interval. Let's see an example of its behavior.

In [5]:
for i in range(population_size):
    individual = binary_individual_spawning_pool.spawn()
    fitness = maximize_ones_fitness(individual)
    print(i, '->', individual, fitness)

0 -> 1101001001111110100000001 0.07142857142857142
1 -> 0011001111101010001110011 0.08333333333333333
2 -> 0001011101101000100100011 0.06666666666666667
3 -> 1110100001100101101101101 0.08333333333333333
4 -> 1011001100110001000100111 0.07142857142857142
5 -> 0000101000010001101010001 0.05555555555555555
6 -> 1111111101000000010011010 0.07692307692307693
7 -> 1101001101111001000001001 0.07142857142857142
8 -> 0011001011001000010011001 0.0625
9 -> 0110100110001100000110010 0.0625


## The stop condition

Now we're gonna specify when our algorithm should stop. This is controlled by a stop condition.

In [6]:
from pynetics.stop import FitnessBound

fitness_stop_condition = FitnessBound(1)

Instances of the class FitnessBound are created by specifying the fitness threshold above which we can stop our algorithm. We have specified a FitnessBound object with a threshold of $1$. That means that all the values below $1$ will not stop our algorithm whereas all the values upper or equal than $1$ will do.

Because our fitness value belongs to the $(0, 1]$ interval, the algorithm will stop only when the population has an individual with a fitness of $1$ (all $1$'s).

## Selecting individuals

For our GA, we're going to use a tournament selection. Tournament selection works by selecting $n$ individuals randomly from the population and then returning the best of them (based on their fitnesses).

In [7]:
from pynetics.selections import Tournament

tournament_selection = Tournament(2)

## Recombining

Now the recombination, i.e. the step where the individuals are selected and their genetic information is inherited by their progeny.

We'll use a `OnePointRecombination`, included in the module `ga_list`. Also, for the recombination we'll specify the probability for two individuals to mate to be 1, that is, they always mate.

In [8]:
from pynetics.ga_list import OnePointRecombination

recombination_probability = 1
recombination = OnePointRecombination()

## Mutations

The same with mutations. The mutation operator we're going to use is `AllGenesCanSwitch`, a mutation where for each binary gene there is a probability to be switched from $0$ to $1$ and viceversa. It belongs to the module `ga_bin`.

In [9]:
from pynetics.ga_bin import AllGenesCanSwitch

mutation_probability = 1 / individual_size
mutation = AllGenesCanSwitch()

## Replacement

Once we've got the offspring, we need to replace the population with these newborns. The operator for that matter will be a `LowElitism` operator, where the worst individuals of the population are replaced by the offspring.

We'll fix the replacement rate in $0.9$, i.e. a $90\%$ of the population will be replaced for each iteration of the loop.

In [10]:
from pynetics.replacements import LowElitism

replacement_rate = 0.9
replacement = LowElitism()

## The algorithm

In [11]:
from pynetics.algorithms import SimpleGA

ga = SimpleGA(
    stop_condition=fitness_stop_condition,
    population_size=population_size,
    fitness=maximize_ones_fitness,
    spawning_pool=binary_individual_spawning_pool,
    selection=tournament_selection,
    recombination=recombination,
    mutation=mutation,
    replacement=replacement,
    p_recombination=recombination_probability,
    p_mutation=mutation_probability,
    replacement_rate=replacement_rate,
)

Now we've created our algorithm, we can run it to find the right solution. Let's see how it works

In [12]:
ga.run()
print(ga.best())

1111111111111111111111111


We can specify functions to be executed while the training takes place. The next example adds some of those functions.

In [13]:
ga.on_start(
    lambda ga: print('Starting genetic algoritm')
).on_end(
    lambda ga: print('Genetic Algorithm ended. Best individual:', ga.best())
).on_step_start(
    lambda ga: print('Step:', ga.generation, '->', end='')
).on_step_end(
    lambda ga: print(ga.best(), 'fitness:', ga.best().fitness())
)
ga.run()

Starting genetic algoritm
Step: 0 ->1011100101101111010001110 fitness: 0.09090909090909091
Step: 1 ->1011111101101111010000110 fitness: 0.1
Step: 2 ->1011111101101111010000110 fitness: 0.1
Step: 3 ->1011101101101111011001110 fitness: 0.1111111111111111
Step: 4 ->1011101101100111111101110 fitness: 0.125
Step: 5 ->1011111101100111111101110 fitness: 0.14285714285714285
Step: 6 ->1011111101100111111101110 fitness: 0.14285714285714285
Step: 7 ->1011111101100111111101110 fitness: 0.14285714285714285
Step: 8 ->1011111101101111111101110 fitness: 0.16666666666666666
Step: 9 ->1011111101101111111101110 fitness: 0.16666666666666666
Step: 10 ->1011111101101111111101110 fitness: 0.16666666666666666
Step: 11 ->1111111111101101111001111 fitness: 0.2
Step: 12 ->1111111111101101111001111 fitness: 0.2
Step: 13 ->1111111111111101111001111 fitness: 0.25
Step: 14 ->1111111111111101111001111 fitness: 0.25
Step: 15 ->1111111111111111111001111 fitness: 0.3333333333333333
Step: 16 ->1111111111111111111001111 f

And here ends the quickstart tutorial!